In [1]:
import json
import os
import itertools
import pandas as pd
import numpy as np
import re
from datetime import datetime as dt
import seaborn as sns

import sys
sys.path.append(".\\sample")
from SALibRepastParams import num_levels, params, random_seed, init_problem, calc_second_order, policies

import batch_data_utils as bd_utils

In [2]:
with open(".//config.json") as f:
    config = json.load(f)

In [3]:
file_datetime_string = config['file_datetime_string']
vehicle_density_timestamp = config['vehicle_density_timestamp']
setting = config['setting']

gis_data_dir = os.path.abspath("..\\data\\model_gis_data")
data_dir = config['batch_data_dir']
img_dir = "..\\output\\img\\"

In [4]:
output_paths = bd_utils.get_ouput_paths(file_datetime_string, vehicle_density_timestamp, data_dir)
output_sd_data = output_paths["output_sd_data"]

In [5]:
output_sd_data

'C:\\Users\\obisargoni\\eclipse-workspace\\repastInterSim\\output\\batch\\model_run_data\\metrics_for_sd_analysis.2022.Apr.21.14_56_46.csv'

In [6]:
dfDD = pd.read_csv(output_sd_data)

## Analyse using tools in ema workbench

# Sensitivity analysis of distance and crossing location entropy outputs

Determining which parameters outcomes are most sensitive to in each policy setting by calculating Sobol indices.

Also evaluating which parameter values lead to which outcome values using a regression tree model. Also demonstrating that parameters have non linear effect on outcomes by attempting to fit linear model.

In [72]:
#from sklearn.model_selection import train_test_split # for splitting the data into train and test samples
from sklearn.metrics import classification_report # for model evaluation metrics
from sklearn import tree, linear_model # for decision tree models
from sklearn.feature_extraction import DictVectorizer

import graphviz # for plotting decision tree graphs

In [73]:
def fitting(X, y, feature_names, criterion, splitter, mdepth, clweight, minleaf):

    # Fit the model
    model = tree.DecisionTreeRegressor(criterion=criterion, 
                                        splitter=splitter, 
                                        max_depth=mdepth,
                                        min_samples_leaf=minleaf, 
                                        random_state=0, 
                                  )
    clf = model.fit(X, y)

    # Predict class labels on training data
    pred_labels_tr = model.predict(X)

    # Tree summary and model evaluation metrics
    print('*************** Tree Summary ***************')
    print('Tree Depth: ', clf.tree_.max_depth)
    print('No. of leaves: ', clf.tree_.n_leaves)
    print('No. of features: ', clf.n_features_in_)
    print('--------------------------------------------------------')
    print("")
    
    print('*************** Evaluation on Training Data ***************')
    score_tr = model.score(X, y)
    print('Accuracy Score: ', score_tr)
    print('--------------------------------------------------------')
    
    # Use graphviz to plot the tree
    dot_data = tree.export_graphviz(clf, out_file=None, 
                                feature_names=feature_names, 
                                class_names=None,
                                filled=True, 
                                rounded=True, 
                                #rotate=True,
                               ) 
    graph = graphviz.Source(dot_data)
    
    # Return relevant data for chart plotting
    return X, y, clf, graph

def fitting_linear_regression(X, y, feature_names):

    # Fit the model
    model = linear_model.LinearRegression( fit_intercept=True)
    clf = model.fit(X, y)
    
    print('*************** Evaluation on Training Data ***************')
    score_tr = model.score(X, y)
    print('Accuracy Score: ', score_tr)
    print('--------------------------------------------------------')
    
    # Return relevant data for chart plotting
    return X, y, clf

In [80]:
# Tree settings
criterion = 'squared_error'
splitter = 'best'
mdepth = 5
clweight = None
minleaf = 10

In [81]:
model_results = {}
grouped = dfDD.groupby([policy_param])
for i, (pv) in enumerate(grouped.groups.keys()):
    print("\nPolicy Param = {}\n".format(pv))
    df_exp = grouped.get_group((pv))
    
    data_dict = df_exp.loc[:, problem['names']].to_dict('records')
    vec = DictVectorizer()  # create the DictVectorizer object
    vec_array = vec.fit_transform(data_dict).toarray()  # execute process on the record dictionaries and transform the result into a numpy array object
    
    X, y, clf, graph = fitting(vec_array, df_exp['cross_entropy'].values, vec.get_feature_names_out(), criterion, splitter, mdepth, clweight, minleaf)
    model_results[pv] = {'X':X, 'y':y, 'clf':clf, 'graph':graph}


Policy Param = False

*************** Tree Summary ***************
Tree Depth:  5
No. of leaves:  14
No. of features:  8
--------------------------------------------------------

*************** Evaluation on Training Data ***************
Accuracy Score:  0.8664076220221705
--------------------------------------------------------

Policy Param = True

*************** Tree Summary ***************
Tree Depth:  5
No. of leaves:  13
No. of features:  8
--------------------------------------------------------

*************** Evaluation on Training Data ***************
Accuracy Score:  0.9524610708916782
--------------------------------------------------------


Inspect the regression tree results for the different policies

In [103]:
graph_informal = model_results[1]['graph']
graph_informal.format='png'
graph_informal.render(filename='tree_cle_informal', directory=img_dir)

'..\\output\\img\\tree_cle_informal.png'

In [104]:
graph_no_informal = model_results[0]['graph']
graph_informal.format='png'
graph_informal.render(filename="tree_cle_no_informal", directory=img_dir)

'..\\output\\img\\tree_cle_no_informal.png'

In [62]:
# compare feature importances, do these match up with sensitivity indices?
clf_inf = model_results[0]['clf']
clf_no_inf = model_results[1]['clf']
fis = [clf_inf.feature_importances_, clf_no_inf.feature_importances_]

dfF = pd.DataFrame(fis, columns = vec.get_feature_names())
dfF

C:\Users\obisargoni\.conda\envs\gforge\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,addPedTicks,addVehicleTicks,alpha,epsilon,lambda,minCrossing,tacticalPlanHorizon,timeThreshold
0,0.0,0.000000,0.084830,0.223881,0.135178,0.000000,0.548266,0.007845
1,0.0,0.001109,0.009338,0.460633,0.159554,0.000343,0.368865,0.000158


Repeat for the distance travelled metric

In [83]:
model_results = {}
grouped = dfDD.groupby([policy_param])
for i, (pv) in enumerate(grouped.groups.keys()):
    print("\nPolicy Param = {}\n".format(pv))
    df_exp = grouped.get_group((pv))
    
    data_dict = df_exp.loc[:, problem['names']].to_dict('records')
    vec = DictVectorizer()  # create the DictVectorizer object
    vec_array = vec.fit_transform(data_dict).toarray()  # execute process on the record dictionaries and transform the result into a numpy array object
    
    X, y, clf, graph = fitting(vec_array, df_exp['DistPAPed'].values, vec.get_feature_names_out(), criterion, splitter, mdepth, clweight, minleaf)
    model_results[pv] = {'X':X, 'y':y, 'clf':clf, 'graph':graph}


Policy Param = False

*************** Tree Summary ***************
Tree Depth:  5
No. of leaves:  11
No. of features:  8
--------------------------------------------------------

*************** Evaluation on Training Data ***************
Accuracy Score:  0.737439419707025
--------------------------------------------------------

Policy Param = True

*************** Tree Summary ***************
Tree Depth:  5
No. of leaves:  12
No. of features:  8
--------------------------------------------------------

*************** Evaluation on Training Data ***************
Accuracy Score:  0.5349548076118277
--------------------------------------------------------


In [84]:
# compare feature importances, do these match up with sensitivity indices?
clf_inf = model_results[0]['clf']
clf_no_inf = model_results[1]['clf']
fis = [clf_inf.feature_importances_, clf_no_inf.feature_importances_]

dfF = pd.DataFrame(fis, columns = vec.get_feature_names_out())
dfF

,addPedTicks,addVehicleTicks,alpha,epsilon,lambda,minCrossing,tacticalPlanHorizon,timeThreshold
0,0.002485,0.0,0.411590,0.003010,0.559996,0.0,0.018366,0.004553
1,0.000000,0.0,0.753566,0.064177,0.145104,0.0,0.000000,0.037153


In [101]:
graph_informal = model_results[1]['graph']
graph_informal.format='png'
graph_informal.render(filename='tree_dist_informal', directory=img_dir)

'..\\output\\img\\tree_dist_informal.png'

In [102]:
graph_no_informal = model_results[0]['graph']
graph_informal.format='png'
graph_informal.render(filename="tree_dist_no_informal", directory=img_dir)

'..\\output\\img\\tree_dist_no_informal.png'

### Linear regression model just to check whether model is non-linear

In [75]:
metrics = ['DistPAPed', 'cross_entropy']
dist_model_results = {}
grouped = dfDD.groupby([policy_param])
for metric in metrics:
    for i, (pv) in enumerate(grouped.groups.keys()):
        print("\n\nMetric = {}".format(metric))
        print("\nPolicy Param = {}".format(pv))
        df_exp = grouped.get_group((pv))
        data_dict = df_exp.loc[:, problem['names']].to_dict('records')
        vec = DictVectorizer()  # create the DictVectorizer object
        vec_array = vec.fit_transform(data_dict).toarray()  # execute process on the record dictionaries and transform the result into a numpy array object
        X, y, clf = fitting_linear_regression(vec_array, df_exp[metric].values, vec.get_feature_names_out())
        dist_model_results[pv] = {'X':X, 'y':y, 'clf':clf, 'graph':graph}
    




Metric = DistPAPed

Policy Param = False
*************** Evaluation on Training Data ***************
Accuracy Score:  0.3239135581711081
--------------------------------------------------------


Metric = DistPAPed

Policy Param = True
*************** Evaluation on Training Data ***************
Accuracy Score:  0.4933043983789418
--------------------------------------------------------


Metric = cross_entropy

Policy Param = False
*************** Evaluation on Training Data ***************
Accuracy Score:  0.5810232299852874
--------------------------------------------------------


Metric = cross_entropy

Policy Param = True
*************** Evaluation on Training Data ***************
Accuracy Score:  0.662619319269176
--------------------------------------------------------


#### Explaining these results

When agetns minimise numbers of crossings get more ordered crossing locations. This can be explained by agents crossing less frequently, specifically it appears that the types or crossings that get cut out are those in mid block locations since these are more avoidable than crossing at the end of a block, which will tend to be in the same location for many agents.

This pattern may not be reproduced for the grid world since in this case entropy is not so sensitive to the MC parameter.